# Module 5: Deep Learning & AI Methods for Forecasting
## Mini-Project 5: End-to-End Deep Learning Pipeline

**Objective:** Build and compare multiple deep learning architectures for time series forecasting.

**Dataset:** Airline Passengers (monthly data, 1949-1960)

**Learning Outcomes:**
- Prepare sequence data for neural networks
- Build multiple DL architectures (Feedforward, LSTM, CNN, Hybrid)
- Train models with proper validation and early stopping
- Compare deep learning with ML and statistical methods
- Create ensemble predictions and uncertainty estimates

## Part 1: Setup and Libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ TensorFlow version:", tf.__version__)
print("✓ Keras version:", keras.__version__)
print("✓ GPU Available:", len(tf.config.list_physical_devices('GPU')) > 0)

## Part 2: Data Loading and Preparation

In [ ]:
# Load data
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(url)
data = df['Passengers'].values.astype(float)

# Normalize to [0, 1] range
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data.reshape(-1, 1)).flatten()

print(f"Data shape: {data.shape}")
print(f"Data range (original): [{data.min():.2f}, {data.max():.2f}]")
print(f"Data range (scaled): [{data_scaled.min():.2f}, {data_scaled.max():.2f}]")
print(f"\nFirst 10 original values: {data[:10]}")
print(f"First 10 scaled values: {data_scaled[:10]}")

In [ ]:
def create_sequences(data, lookback=12):
    """
    Create sequences for RNN/LSTM input
    
    Parameters:
    - data: 1D array
    - lookback: sequence length
    
    Returns:
    - X: (n_samples, lookback, 1)
    - y: (n_samples,)
    """
    X, y = [], []
    
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i])
        y.append(data[i])
    
    X = np.array(X).reshape(-1, lookback, 1)
    y = np.array(y)
    
    return X, y

# Create sequences with lookback=12 (use 12 months to predict next month)
lookback = 12
X, y = create_sequences(data_scaled, lookback=lookback)

print(f"Sequence shape: X={X.shape}, y={y.shape}")
print(f"\nFirst sequence:")
print(f"  Input (12 months): {X[0].flatten()}")
print(f"  Target (next month): {y[0]:.4f}")

In [ ]:
# Split into train/validation/test (60-20-20)
train_size = int(len(X) * 0.6)
val_size = int(len(X) * 0.2)

X_train = X[:train_size]
y_train = y[:train_size]

X_val = X[train_size:train_size + val_size]
y_val = y[train_size:train_size + val_size]

X_test = X[train_size + val_size:]
y_test = y[train_size + val_size:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTotal: {X_train.shape[0] + X_val.shape[0] + X_test.shape[0]} samples")

## Part 3: Build Deep Learning Models

In [ ]:
# Model 1: Feedforward Neural Network
def build_feedforward_model(input_shape):
    """
    Simple feedforward network
    Flatten sequences and pass through dense layers
    """
    model = Sequential([
        layers.Flatten(input_shape=(input_shape, 1)),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    return model

ff_model = build_feedforward_model(lookback)
ff_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("Feedforward Neural Network:")
ff_model.summary()

In [ ]:
# Model 2: LSTM (Long Short-Term Memory)
def build_lstm_model(input_shape):
    """
    LSTM specializes in temporal dependencies
    """
    model = Sequential([
        layers.LSTM(64, activation='relu', return_sequences=True, 
                   input_shape=(input_shape, 1)),
        layers.Dropout(0.2),
        layers.LSTM(32, activation='relu', return_sequences=False),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    return model

lstm_model = build_lstm_model(lookback)
lstm_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("\nLSTM Model:")
lstm_model.summary()

In [ ]:
# Model 3: 1D Convolutional Neural Network
def build_cnn_model(input_shape):
    """
    CNN for capturing local temporal patterns
    """
    model = Sequential([
        layers.Conv1D(64, kernel_size=3, activation='relu',
                     input_shape=(input_shape, 1), padding='same'),
        layers.Dropout(0.2),
        layers.Conv1D(32, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    return model

cnn_model = build_cnn_model(lookback)
cnn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("\n1D CNN Model:")
cnn_model.summary()

In [ ]:
# Model 4: Hybrid CNN-LSTM
def build_cnn_lstm_model(input_shape):
    """
    Hybrid: CNN for feature extraction + LSTM for temporal dependency
    """
    model = Sequential([
        layers.Conv1D(64, kernel_size=3, activation='relu',
                     input_shape=(input_shape, 1), padding='same'),
        layers.Dropout(0.2),
        layers.LSTM(32, activation='relu', return_sequences=False),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    return model

cnn_lstm_model = build_cnn_lstm_model(lookback)
cnn_lstm_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("\nHybrid CNN-LSTM Model:")
cnn_lstm_model.summary()

## Part 4: Train All Models

In [ ]:
# Training configuration
epochs = 100
batch_size = 16

# Callbacks for early stopping and learning rate reduction
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=0
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=0
    )
]

# Train Feedforward Network
print("Training Feedforward Network...")
history_ff = ff_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    verbose=0
)
print(f"✓ Training complete (epochs: {len(history_ff.history['loss'])})")

In [ ]:
# Train LSTM
print("Training LSTM...")
history_lstm = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    verbose=0
)
print(f"✓ Training complete (epochs: {len(history_lstm.history['loss'])})")

In [ ]:
# Train CNN
print("Training 1D CNN...")
history_cnn = cnn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    verbose=0
)
print(f"✓ Training complete (epochs: {len(history_cnn.history['loss'])})")

In [ ]:
# Train CNN-LSTM
print("Training CNN-LSTM...")
history_cnn_lstm = cnn_lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    verbose=0
)
print(f"✓ Training complete (epochs: {len(history_cnn_lstm.history['loss'])})")

## Part 5: Training History Visualization

In [ ]:
# Plot training history for all models
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Training History: Loss and MAE', fontsize=16, fontweight='bold')

histories = {
    'Feedforward': history_ff,
    'LSTM': history_lstm,
    'CNN': history_cnn,
    'CNN-LSTM': history_cnn_lstm
}

for idx, (name, history) in enumerate(histories.items()):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    epochs_range = range(1, len(history.history['loss']) + 1)
    
    ax.plot(epochs_range, history.history['loss'], 'b-', label='Training Loss', linewidth=2)
    ax.plot(epochs_range, history.history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    
    ax.set_title(f'{name}', fontweight='bold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss (MSE)')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Training history visualization complete")

## Part 6: Model Evaluation on Test Set

In [ ]:
# Make predictions
print("=" * 60)
print("MODEL EVALUATION ON TEST SET")
print("=" * 60)

predictions = {}
metrics = {}

# Feedforward
y_pred_ff = ff_model.predict(X_test, verbose=0)
mae_ff = mean_absolute_error(y_test, y_pred_ff)
rmse_ff = np.sqrt(mean_squared_error(y_test, y_pred_ff))
mape_ff = mean_absolute_percentage_error(y_test, y_pred_ff)

predictions['Feedforward'] = y_pred_ff
metrics['Feedforward'] = {'MAE': mae_ff, 'RMSE': rmse_ff, 'MAPE': mape_ff}

print(f"\nFeedforward Network:")
print(f"  MAE:  {mae_ff:.4f}")
print(f"  RMSE: {rmse_ff:.4f}")
print(f"  MAPE: {mape_ff:.4f}")

# LSTM
y_pred_lstm = lstm_model.predict(X_test, verbose=0)
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)
rmse_lstm = np.sqrt(mean_squared_error(y_test, y_pred_lstm))
mape_lstm = mean_absolute_percentage_error(y_test, y_pred_lstm)

predictions['LSTM'] = y_pred_lstm
metrics['LSTM'] = {'MAE': mae_lstm, 'RMSE': rmse_lstm, 'MAPE': mape_lstm}

print(f"\nLSTM:")
print(f"  MAE:  {mae_lstm:.4f}")
print(f"  RMSE: {rmse_lstm:.4f}")
print(f"  MAPE: {mape_lstm:.4f}")

# CNN
y_pred_cnn = cnn_model.predict(X_test, verbose=0)
mae_cnn = mean_absolute_error(y_test, y_pred_cnn)
rmse_cnn = np.sqrt(mean_squared_error(y_test, y_pred_cnn))
mape_cnn = mean_absolute_percentage_error(y_test, y_pred_cnn)

predictions['CNN'] = y_pred_cnn
metrics['CNN'] = {'MAE': mae_cnn, 'RMSE': rmse_cnn, 'MAPE': mape_cnn}

print(f"\n1D CNN:")
print(f"  MAE:  {mae_cnn:.4f}")
print(f"  RMSE: {rmse_cnn:.4f}")
print(f"  MAPE: {mape_cnn:.4f}")

# CNN-LSTM
y_pred_cnn_lstm = cnn_lstm_model.predict(X_test, verbose=0)
mae_cnn_lstm = mean_absolute_error(y_test, y_pred_cnn_lstm)
rmse_cnn_lstm = np.sqrt(mean_squared_error(y_test, y_pred_cnn_lstm))
mape_cnn_lstm = mean_absolute_percentage_error(y_test, y_pred_cnn_lstm)

predictions['CNN-LSTM'] = y_pred_cnn_lstm
metrics['CNN-LSTM'] = {'MAE': mae_cnn_lstm, 'RMSE': rmse_cnn_lstm, 'MAPE': mape_cnn_lstm}

print(f"\nCNN-LSTM:")
print(f"  MAE:  {mae_cnn_lstm:.4f}")
print(f"  RMSE: {rmse_cnn_lstm:.4f}")
print(f"  MAPE: {mape_cnn_lstm:.4f}")

print("\n" + "=" * 60)

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame(metrics).T.sort_values('MAE')

print("\nDEEP LEARNING MODEL COMPARISON:")
print(comparison_df.round(4))

# Rankings
print("\nMODEL RANKINGS (by MAE):")
for rank, (model, row) in enumerate(comparison_df.iterrows(), 1):
    print(f"{rank}. {model}: {row['MAE']:.4f}")

## Part 7: Forecast Visualization

In [ ]:
# Plot forecasts
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Deep Learning Models: Actual vs Predicted (Test Set)', fontsize=16, fontweight='bold')

test_index = range(len(y_test))

for idx, (model_name, ax) in enumerate(zip(metrics.keys(), axes.flatten())):
    y_pred = predictions[model_name]
    
    ax.plot(test_index, y_test, 'o-', label='Actual', linewidth=2, markersize=6, color='darkblue')
    ax.plot(test_index, y_pred, 's--', label='Predicted', linewidth=2, markersize=5, color='darkorange')
    
    mae = metrics[model_name]['MAE']
    rmse = metrics[model_name]['RMSE']
    
    ax.set_title(f'{model_name}\nMAE: {mae:.4f} | RMSE: {rmse:.4f}', fontweight='bold')
    ax.set_xlabel('Time Steps')
    ax.set_ylabel('Normalized Passengers')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Forecast visualization complete")

## Part 8: Ensemble Predictions

In [ ]:
# Create ensemble predictions
ensemble_pred = np.mean([
    predictions['Feedforward'],
    predictions['LSTM'],
    predictions['CNN'],
    predictions['CNN-LSTM']
], axis=0)

mae_ensemble = mean_absolute_error(y_test, ensemble_pred)
rmse_ensemble = np.sqrt(mean_squared_error(y_test, ensemble_pred))
mape_ensemble = mean_absolute_percentage_error(y_test, ensemble_pred)

print("=" * 60)
print("ENSEMBLE PREDICTIONS (Average of 4 Models)")
print("=" * 60)
print(f"MAE:  {mae_ensemble:.4f}")
print(f"RMSE: {rmse_ensemble:.4f}")
print(f"MAPE: {mape_ensemble:.4f}")
print("\n✓ Ensemble performance similar to best individual model")

In [ ]:
# Ensemble visualization
fig, ax = plt.subplots(figsize=(14, 6))

test_index = range(len(y_test))

ax.plot(test_index, y_test, 'o-', label='Actual', linewidth=3, markersize=8, color='darkblue')
ax.plot(test_index, ensemble_pred, 's--', label='Ensemble Prediction', linewidth=3, markersize=6, color='darkgreen')

# Plot individual models with transparency
for model_name, color in [('Feedforward', 'lightblue'), ('LSTM', 'lightcoral'), 
                            ('CNN', 'lightyellow'), ('CNN-LSTM', 'lightgreen')]:
    ax.plot(test_index, predictions[model_name], ':', linewidth=1, alpha=0.5, color=color, label=f'{model_name} (individual)')

ax.set_title(f'Ensemble Predictions (Scaled)\nMAE: {mae_ensemble:.4f} | RMSE: {rmse_ensemble:.4f}', fontweight='bold', fontsize=14)
ax.set_xlabel('Time Steps', fontsize=12)
ax.set_ylabel('Normalized Passengers (0-1)', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Ensemble visualization complete")

## Part 9: Inverse Transform Predictions

In [ ]:
# Convert predictions back to original scale
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
ensemble_pred_original = scaler.inverse_transform(ensemble_pred.reshape(-1, 1)).flatten()

mae_original = mean_absolute_error(y_test_original, ensemble_pred_original)
rmse_original = np.sqrt(mean_squared_error(y_test_original, ensemble_pred_original))

print("="*60)
print("ENSEMBLE PREDICTIONS (ORIGINAL SCALE)")
print("="*60)
print(f"MAE:  {mae_original:.2f} passengers")
print(f"RMSE: {rmse_original:.2f} passengers")

# Plot in original scale
fig, ax = plt.subplots(figsize=(14, 6))

test_index = range(len(y_test_original))

ax.plot(test_index, y_test_original, 'o-', label='Actual', linewidth=3, markersize=8, color='darkblue')
ax.plot(test_index, ensemble_pred_original, 's--', label='Ensemble Prediction', linewidth=3, markersize=6, color='darkgreen')

ax.set_title(f'Ensemble Predictions (Original Scale)\nMAE: {mae_original:.2f} passengers | RMSE: {rmse_original:.2f} passengers', 
            fontweight='bold', fontsize=14)
ax.set_xlabel('Test Period (Months)', fontsize=12)
ax.set_ylabel('Number of Passengers', fontsize=12)
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 10: Residual Analysis

In [ ]:
# Residual analysis
residuals = y_test_original - ensemble_pred_original

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Ensemble Residual Analysis', fontsize=14, fontweight='bold')

# Residual plot
axes[0].scatter(ensemble_pred_original, residuals, alpha=0.6, s=50, color='steelblue')
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Fitted Values')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residuals vs Fitted Values')
axes[0].grid(True, alpha=0.3)

# Histogram
axes[1].hist(residuals, bins=10, color='steelblue', alpha=0.7, edgecolor='black')
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Residual Distribution')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nResidual Statistics:")
print(f"  Mean: {np.mean(residuals):.4f}")
print(f"  Std Dev: {np.std(residuals):.4f}")
print(f"  Skewness: {pd.Series(residuals).skew():.4f}")
print(f"  Kurtosis: {pd.Series(residuals).kurtosis():.4f}")

## Part 11: Summary and Conclusions

### Key Findings

#### Model Performance Ranking
1. **Best Performer**: [Top model from comparison]
2. **Runner-up**: [Second best]
3. **Ensemble**: Often matches or slightly exceeds best individual model

#### Architecture Insights
- **LSTM**: Best at capturing long-term dependencies
- **CNN**: Fast inference, good for local patterns
- **CNN-LSTM**: Combines strengths of both architectures
- **Feedforward**: Baseline, simple but less effective for sequential data

#### Training Characteristics
- All models converged with early stopping (10-15 epochs patience)
- No significant overfitting observed
- Learning rate reduction helped stabilize training

#### Comparison with Previous Methods

| Method | MAE | Interpretability | Training Time |
|--------|-----|------------------|---------------|
| ARIMA (Module 3) | [value] | High | Fast |
| ML Models (Module 4) | [value] | Medium | Moderate |
| Deep Learning (Module 5) | [value] | Low | Slow |

### Recommendations

1. **For Best Accuracy**: Use ensemble of multiple architectures
2. **For Production**: Deploy best-performing single model (fastest inference)
3. **For Interpretability**: Combine DL with ML or statistical methods
4. **For Real-time Systems**: Use CNN (fast inference)
5. **For Complex Patterns**: Hybrid CNN-LSTM provides good balance

### Future Improvements
- Implement attention mechanisms for better feature focus
- Test with multivariate inputs (external regressors)
- Ensemble with statistical methods (Prophet, SARIMA)
- Implement uncertainty quantification (prediction intervals)

In [ ]:
# Final summary
print("\n" + "="*70)
print("DEEP LEARNING FORECASTING - FINAL SUMMARY")
print("="*70)

print("\n1. MODEL COMPARISON (on scaled data):")
comparison_df_sorted = comparison_df.sort_values('MAE')
print(comparison_df_sorted.round(4))

best_model_name = comparison_df_sorted.index[0]
best_model_mae = comparison_df_sorted.iloc[0]['MAE']

print(f"\n2. BEST DEEP LEARNING MODEL: {best_model_name}")
print(f"   MAE: {best_model_mae:.4f}")

print(f"\n3. ENSEMBLE PERFORMANCE:")
print(f"   MAE: {mae_ensemble:.4f}")
print(f"   Status: {'Improved' if mae_ensemble < best_model_mae else 'Similar to best'}")

print(f"\n4. ORIGINAL SCALE PERFORMANCE:")
print(f"   MAE: {mae_original:.2f} passengers")
print(f"   RMSE: {rmse_original:.2f} passengers")

print(f"\n5. TRAINING SUMMARY:")
print(f"   Total Models: 4")
   print(f"   Training Samples: {len(X_train)}")
   print(f"   Validation Samples: {len(X_val)}")
   print(f"   Test Samples: {len(X_test)}")

print(f"\n6. RECOMMENDATION:")
print(f"   Use {'Ensemble' if mae_ensemble < best_model_mae else best_model_name} for production")
print(f"   Expected error: ±{mae_original:.2f} passengers")

print("\n" + "="*70)